# Estimating Coordinated Daily Activity Patterns

This notebook illustrates how to re-estimate ActivitySim's CDAP model. This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [114]:
import numpy as np
import pandas as pd
import larch
import os

pd.options.display.max_columns = 150

In [115]:
os.chdir('C:\ABM3_dev\outputs')

In [116]:
cdap_dir = r'output\estimation_data_bundle\cdap'
full_edb = pd.read_csv(os.path.join(cdap_dir, 'cdap_values_combined_full.csv'))

C:\Users\davidh\AppData\Local\Temp\2\ipykernel_32752\2743892615.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  full_edb = pd.read_csv(os.path.join(cdap_dir, 'cdap_values_combined_full.csv'))


In [360]:
print("Number of total people requiring cdap:", len(full_edb))
# df = full_edb[full_edb.hhsize <= 10]
df = full_edb
print("Number of people with hhsize <= 2:", len(df))

Number of total people requiring cdap: 96881
Number of people with hhsize <= 2: 96881


In [361]:
# Creating expressions that need to use numpy 
df['tmp_ptype3_school_mc_logsum'] = np.where((df.ptype == 3)  & (df.school_modechoice_logsum > 0), df.school_modechoice_logsum, 0)
df['tmp_has_below_avg_shop_accessibility'] = (df.shopping_accessibility < df.shopping_accessibility.mean())

In [362]:
df['tmp_has_below_avg_shop_accessibility'].value_counts(dropna=False)

True     48505
False    48376
Name: tmp_has_below_avg_shop_accessibility, dtype: int64

In [363]:
df.to_csv(os.path.join(cdap_dir, 'cdap_values_combined.csv'), index=False)

In [358]:
spec = pd.read_csv(os.path.join(cdap_dir, 'cdap_INDIV_AND_HHSIZE1_SPEC.csv'))
coeffs = spec[['M', 'N', 'H']].melt(value_name='coefficient_name')
coeffs = coeffs[~coeffs.coefficient_name.isna()].drop(columns='variable').drop_duplicates(subset='coefficient_name', keep='first')
coeffs['value'] = 0.0
coeffs['constrain'] = 'F'
coeffs.set_index('coefficient_name', inplace=True)
coeffs.loc['coef_UNAVAILABLE'] = (-999.0, 'T')
coeffs.to_csv(os.path.join(cdap_dir, 'coef_helper.csv'))

# Load data and prep model for estimation

In [364]:
modelname = "cdap"

# import importlib
# importlib.reload(activitysim.estimation.larch)
from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

c:\abm3_dev\activitysim\activitysim\estimation\larch\cdap.py:375: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)


Including joint tour utiltiy?: False


c:\abm3_dev\activitysim\activitysim\estimation\larch\cdap.py:375: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>


# Review Inputs

## Settings

In [365]:
data.settings

┣          INDIV_AND_HHSIZE1_SPEC: 'cdap_indiv_and_hhsize1.csv'
┣                    COEFFICIENTS: 'cdap_coefficients.csv'
┣        INTERACTION_COEFFICIENTS: 'cdap_interaction_coefficients.csv'
┣ FIXED_RELATIVE_PROPORTIONS_SPEC: 'cdap_fixed_relative_proportions.csv'
┣          ADD_JOINT_TOUR_UTILITY: True
┣         JOINT_TOUR_COEFFICIENTS: 'cdap_joint_tour_coefficients.csv'
┣       JOINT_TOUR_USEFUL_COLUMNS: ---
┃                                  - auto_ownership
┃                                  - income
┃                                  - num_workers
┃                                  - shopping_accessibility
┃                                  - workplace_modechoice_logsum
┃                                  ...
┣                       CONSTANTS: ┣       FULL: 1
┃                                  ┣       PART: 2
┃                                  ┣ UNIVERSITY: 3
┃                                  ┣    NONWORK: 4
┃                                  ┣    RETIRED: 5
┃                   

## Households

In [366]:
data.households

,household_id,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,children,day,bldgsz,res_type,transponder_ownership,survey_year,day.1,HH_ID,chauf_id1,chauf_id2,child_id1,child_id2,child_id3,school_escorting_outbound,school_escorting_inbound,school_escorting_outbound_cond,joint_tour_frequency,joint_tour_frequency_composition,has_joint_tour,sample_rate,income_in_thousands,income_segment,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_6_to_12,num_children_5_to_15,num_children_16_to_17,num_gradeschool,num_highschool,num_college_age,num_young_adults,num_predrive_child,num_nonworker_adults,num_university_students,num_fullTime_workers,num_partTime_workers,num_retired_adults,num_highschool_graduates,num_children_6_to_15,num_young_retirees,num_old_retirees,non_family,family,home_is_urban,home_is_rural,num_hh_in_zone,av_ownership,workplace_location_accessibility,shopping_accessibility,othdiscr_accessibility,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,auPkRetail,auPkTotal,auOpRetail,auOpTotal,trPkRetail,trPkTotal,trPkHH,...,emp_state_local_gov_blue,emp_state_local_gov_white,emp_public_ed,emp_own_occ_dwell_mgmt,emp_fed_gov_accts,emp_st_lcl_gov_accts,emp_cap_accts,emp_total,enrollgradekto8,enrollgrade9to12,collegeenroll,othercollegeenroll,adultschenrl,ech_dist,hch_dist,pseudomsa,parkarea,hstallsoth,hstallssam,hparkcost,numfreehrs,dstallsoth,dstallssam,dparkcost,mstallsoth,mstallssam,mparkcost,parkactive,openspaceparkpreserve,beachactive,budgetroom,economyroom,luxuryroom,midpriceroom,upscaleroom,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations,totint,duden,empden,popden,retempden,totintbin,empdenbin,dudenbin,PopEmpDenPerMi,TAZ,poe_id,external_work,external_nonwork,external_TAZ,external_MAZ,walk_dist_local_bus,walk_dist_premium_transit,ML_DIST,AVGTTS,PCTDETOUR,terminal_time,household_density,population_density,employment_density,density_index,is_cbd,tot_collegeenroll,preschool_target,is_parking_zone,num_hh_joint_tours
0,1,13119,44996.25,4,1,2,1,2,1,2,1,False,2016,1,161000385,2.0,1.0,3.0,4.0,NaN,1,1,1,0_tours,0.0,0.0,1,44.99625,2,3,2,2,2,0,1,2,0,2,1,0,0,2,1,0,0,1,0,2,2,0,0,False,True,False,False,14,False,10.800842,11.217657,14.884606,0.0,0,4,2,0,2,4,False,9.114823,11.525027,9.528020,11.959262,8.057191,10.158213,9.705998,...,0,0,0,0,0,0,0,4,0,0,0,0,0,432801,432801,3,4,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0,0,0,0,0,0,0,1,4,7.8278,22.4448,22.4448,22.4448,120,0,0,46,4.2780,0.1737,12.2946,0.0000,1,1,1,7979.7289,2439,NaN,0.0,0.0,NaN,NaN,9999.000,9999.000,0.4980,1.52527,0.09832,0.0,4.856359,13.856216,0.178215,0.171907,False,0,315,True,0
1,2,19303,38685.00,2,1,2,0,0,1,2,1,False,2016,1,161000451,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0.0,0.0,1,38.68500,2,2,2,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,1,0,False,True,False,False,7,False,10.394149,11.339877,13.464349,0.0,0,2,2,0,0,2,False,8.845941,11.046439,9.132367,11.360222,5.187108,7.315624,7.712888,...,0,0,0,0,0,0,0,10,0,0,0,0,0,432901,432901,7,4,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0,0,0,0,0,0,0,1,1,7.2567,33.7884,33.7884,33.7884,120,0,0,11,2.1970,2.5400,6.8756,1.2043,1,1,1,6025.9783,1260,NaN,0.0,0.0,NaN,NaN,0.560,1.000,7.2306,0.16124,0.20751,0.0,1.302222,4.321010,0.295960,0.241152,False,0,156,True,0
2,3,12446,17175.00,1,4,1,0,0,1,8,4,False,2016,1,161000525,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0.0,0.0,1,17.17500,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,True,False,False,False,14,False,11.403241,12.191229,14.323247,0.0,0,1,1,0,0,1,False,9.623196,11.774019,9.884143,12.113552,6.207654,8.333393,8.881074,...,0,0,0,0,0,0,0,5,0,0,0,0,0,411201,423101,5,4,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0,0,0,0,0,0,0,1,13,11.4048,8.3940,8.3940,8.3940,120,0,0,40,5.4644,9.8891,16.0063,4.2917,1,1,2,16573.0548,4167,NaN,0.0,0.0,NaN,NaN,0.18

## Person Data

In [367]:
data.person_data

,person_id,model_choice,override_choice,household_id,PNUM,age,sex,pemploy,pstudent,is_student,ptype,school_zone_id,workplace_zone_id,free_parking_at_work,work_from_home,telecommute_frequency,day,educ,external_worker_identification,external_workplace_zone_id,transit_pass_subsidy,transit_pass_ownership,industry,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_eatout,_social,_othdiscr,non_mandatory_tour_frequency,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_preschool,is_gradeschool,is_highschool,is_university,school_segment,is_worker,is_fulltime_worker,is_parttime_worker,is_internal_worker,is_external_worker,home_zone_id,time_factor_work,time_factor_nonwork,is_income_less25K,is_income_25K_to_60K,is_income_60K_to_120K,is_income_greater60K,is_income_greater120K,is_non_worker_in_HH,is_all_adults_full_time_workers,is_pre_drive_child_in_HH,is_out_of_home_worker,external_workplace_location_logsum,external_workplace_modechoice_logsum,school_location_logsum,school_modechoice_logsum,distance_to_school,roundtrip_auto_time_to_school,...,emp_own_occ_dwell_mgmt,emp_fed_gov_accts,emp_st_lcl_gov_accts,emp_cap_accts,emp_total,enrollgradekto8,enrollgrade9to12,collegeenroll,othercollegeenroll,adultschenrl,ech_dist,hch_dist,pseudomsa,parkarea,hstallsoth,hstallssam,hparkcost,numfreehrs,dstallsoth,dstallssam,dparkcost,mstallsoth,mstallssam,mparkcost,parkactive,openspaceparkpreserve,beachactive,budgetroom,economyroom,luxuryroom,midpriceroom,upscaleroom,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations,totint,duden,empden,popden,retempden,totintbin,empdenbin,dudenbin,PopEmpDenPerMi,TAZ,poe_id,external_work,external_nonwork,external_TAZ,external_MAZ,walk_dist_local_bus,walk_dist_premium_transit,ML_DIST,AVGTTS,PCTDETOUR,terminal_time,household_density,population_density,employment_density,density_index,is_cbd,tot_collegeenroll,preschool_target,is_parking_zone,chunk_id,tmp_ptype3_school_mc_logsum,tmp_has_below_avg_shop_accessibility,cdap_rank
0,1,M,N,1,1,46,2,2,3,False,2,-1,5476,True,False,No_Telecommute,1,13,1,-1,0,0,healthcare,N,NaN,1,0,0,1,0,1,96,False,True,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,0,True,False,True,True,False,13119,0.407362,0.648903,False,True,False,False,False,True,False,True,True,NaN,NaN,NaN,NaN,NaN,0.000000,...,0,0,0,0,4,0,0,0,0,0,432801,432801,3,4,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0,0,0,0,0,0,0,1,4,7.8278,22.4448,22.4448,22.4448,120,0,0,46,4.2780,0.1737,12.2946,0.0000,1,1,1,7979.7289,2439,NaN,0.0,0.0,NaN,NaN,9999.000,9999.000,0.4980,1.52527,0.09832,0.0,4.856359,13.856216,0.178215,0.171907,False,0,315,True,0,0.0,True,1
1,2,N,N,1,2,53,1,3,3,False,4,-1,-1,False,False,NaN,1,13,1,-1,0,0,NaN,N,NaN,0,0,0,0,0,1,1,False,True,True,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,0,False,False,False,False,False,13119,0.279769,0.612654,False,True,False,False,False,True,False,True,False,NaN,NaN,NaN,NaN,NaN,0.000000,...,0,0,0,0,4,0,0,0,0,0,432801,432801,3,4,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0,0,0,0,0,0,0,1,4,7.8278,22.4448,22.4448,22.4448,120,0,0,46,4.2780,0.1737,12.2946,0.0000,1,1,1,7979.7289,2439,NaN,0.0,0.0,NaN,NaN,9999.000,9999.000,0.4980,1.52527,0.09832,0.0,4.856359,13.856216,0.178215,0.171907,False,0,315,True,0,0.0,True,4
2,3,M,M,1,3,8,2,4,1,True,7,13464,-1,False,False,NaN,1,0,1,-1,0,0,NaN,M,school1,0,0,0,0,0,0,0,False,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,True,False,False,2,False,False,False,False,False,13119,0.412113,1.453034,False,True,False,False,False,True,False,True,False,NaN,NaN,0.853976,0.853976,3.675232,23.554642,...,0,0,0,0,4,0,0,0,0,0,432801,432801,3,4,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0,0,0,0,0,0,0,1,4,7.8278,22.44

In [368]:
(data.person_data.override_choice == data.person_data.cdap_activity).all()

True

In [369]:
df = data.person_data.copy()
df['mand_status'] = np.select(condlist=[df.is_worker, df.is_student, (df.is_worker & df.is_student), ~(df.is_worker | df.is_student)], choicelist=['worker', 'student', 'both', 'neither'])
pd.crosstab(df.mand_status, df.override_choice, margins=True)

override_choice,H,M,N,All
mand_status,,,,
neither,22421,0,6641,29062
student,13345,2346,2356,18047
worker,33073,9458,7241,49772
All,68839,11804,16238,96881


In [370]:
pd.crosstab(df.telecommute_frequency, df.override_choice, margins=True)

override_choice,H,M,N,All
telecommute_frequency,,,,
1_day_week,3092,1473,562,5127
2_3_days_week,1380,309,385,2074
4_days_week,414,204,354,972
No_Telecommute,19710,7409,3011,30130
All,24596,9395,4312,38303


## interaction_coef

In [371]:
data.interaction_coef

,activity,interaction_ptypes,coefficient,cardinality,slug
0,H,11,0,2,HH11
1,H,12,0,2,HH12
2,H,13,0,2,HH13
3,H,14,0,2,HH14
4,H,15,0,2,HH15
5,H,16,0,2,HH16
6,H,17,0,2,HH17
7,H,18,0,2,HH18
8,H,22,0,2,HH22
9,H,23,0,2,HH23


## Spec 1

In [372]:
data.spec1

,Description,Expression,M,N,H
0,Full-time worker ASC,ptype == 1,coef_full_time_worker_asc_M,coef_full_time_worker_asc_N,NaN
1,Part-time worker ASC,ptype == 2,coef_part_time_worker_asc_M,coef_part_time_worker_asc_N,NaN
2,University student ASC,ptype == 3,coef_university_student_asc_M,coef_university_student_asc_N,NaN
3,Non-working adult ASC,ptype == 4,NaN,coef_non_working_adult_asc_N,NaN
4,Retired ASC,ptype == 5,NaN,coef_retired_asc_N,NaN
5,Driving-age child who is in school ASC,ptype == 6,coef_driving_age_child_asc_M,coef_driving_age_child_asc_N,NaN
6,Pre-driving-age child who is in school ASC,ptype == 7,coef_pre_driving_age_child_asc_M,coef_pre_driving_age_child_asc_N,NaN
7,Preschool child ASC,ptype == 8,coef_preschool_child_asc_M,coef_preschool_child_asc_N,NaN
8,Full-time worker interaction with female gender,(ptype == 1) & (sex == 2),coef_full_time_worker_female_gender_M,coef_full_time_worker_female_gender_N,NaN
9,Part-time worker interaction with female gender,(ptype == 2) & (sex == 2),coef_part_time_worker_female_gender_M,coef_part_time_worker_female_gender_N,NaN


# Review Model

## Utility Functions for HHSize 1

In [373]:
model[0].utility_functions() ## HH Size = 1 

alt,formula
1,+ P.coef_full_time_worker_asc_M * X('ptype == 1') + P.coef_part_time_worker_asc_M * X('ptype == 2') + P.coef_university_student_asc_M * X('ptype == 3') + P.coef_driving_age_child_asc_M * X('ptype == 6') + P.coef_pre_driving_age_child_asc_M * X('ptype == 7') + P.coef_preschool_child_asc_M * X('ptype == 8') + P.coef_full_time_worker_female_gender_M * X('(ptype == 1) & (sex == 2)') + P.coef_part_time_worker_female_gender_M * X('(ptype == 2) & (sex == 2)') + P.coef_university_student_female_gender_M * X('(ptype == 3) & (sex == 2)') + P.coef_retired_female_M * X('(ptype == 5) & (sex == 2)') + P.coef_driving_age_child_more_cars_than_workers_M * X('(ptype == 6) & (auto_ownership > num_workers)') + P.coef_full_time_worker_fewer_cars_than_workers_M * X('(ptype == 1) & (auto_ownership < num_workers)') + P.coef_full_time_worker_income_less_than_30k_M * X('(ptype == 1) & (income < 30000)') + P.coef_part_time_worker_income_less_than_30k_M * X('(ptype == 2) & (income < 30000)') + P.coef_preschool_child_income_between_60k_and_100k_M * X('(ptype == 8) & (income >= 60000) & (income <= 100000)') + P.coef_UNAVAILABLE * X('(is_student == False) & (is_worker == False)')
2,+ P.coef_full_time_worker_asc_N * X('ptype == 1') + P.coef_part_time_worker_asc_N * X('ptype == 2') + P.coef_university_student_asc_N * X('ptype == 3') + P.coef_non_working_adult_asc_N * X('ptype == 4') + P.coef_retired_asc_N * X('ptype == 5') + P.coef_driving_age_child_asc_N * X('ptype == 6') + P.coef_pre_driving_age_child_asc_N * X('ptype == 7') + P.coef_preschool_child_asc_N * X('ptype == 8') + P.coef_full_time_worker_female_gender_N * X('(ptype == 1) & (sex == 2)') + P.coef_part_time_worker_female_gender_N * X('(ptype == 2) & (sex == 2)') + P.coef_university_student_female_gender_N * X('(ptype == 3) & (sex == 2)') + P.coef_non_working_adult_female_gender_N * X('(ptype == 4) & (sex == 2)') + P.coef_retired_female_N * X('(ptype == 5) & (sex == 2)') + P.coef_driving_age_child_female_N * X('(ptype == 6) & (sex == 2)') + P.coef_non_working_adult_more_cars_than_workers_N * X('(ptype == 4) & (auto_ownership > num_workers)') + P.coef_retired_more_cars_than_workers_N * X('(ptype == 5) & (auto_ownership > num_workers)') + P.coef_full_time_worker_retail_accessibility_N * X('(ptype == 1) & tmp_has_below_avg_shop_accessibility') + P.coef_ft_worker_work_from_home_N * X('(ptype == 1) & (work_from_home)') + P.coef_pt_worker_work_from_home_N * X('(ptype == 2) & (work_from_home)') + P.coef_telecommute_1_day_week_N * X('telecommute_frequency=='1_day_week'') + P.coef_telecommute_2_3_days_week_N * X('telecommute_frequency=='2_3_days_week'') + P.coef_telecommute_4_days_week_N * X('telecommute_frequency=='4_days_week'')
3,+ P.coef_full_time_worker_income_less_than_30k_H * X('(ptype == 1) & (income < 30000)') + P.coef_university_student_income_between_60k_and_100k_H * X('(ptype == 3) & (income >= 60000) & (income <= 100000)') + P.coef_non_working_adult_income_more_than_100k_H * X('(ptype == 4) & (income >= 100000)') + P.coef_retired_income_more_than_100k_H * X('(ptype == 5) & (income >= 100000)') + P.coef_telecommute_1_day_week_H * X('telecommute_frequency=='1_day_week'') + P.coef_telecommute_2_3_days_week_H * X('telecommute_frequency=='2_3_days_week'') + P.coef_telecommute_4_days_week_H * X('telecommute_frequency=='4_days_week'')


## Utility Functions for HHSize 2

In [375]:
# model[1].utility_functions() ## HH Size = 2

# Re-Estimate Model

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [376]:
# r = model.estimate(method='BHHH', options={'maxiter':1000})
model.estimate(method='BHHH', options={'maxiter':1500})

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_UNAVAILABLE,-999.000000,-999.0,0.0,-999.0,-999.0,1,,-999.000000
coef_driving_age_child_asc_M,-2.274645,0.0,0.0,NaN,NaN,0,,-2.274645
coef_driving_age_child_asc_N,-1.886586,0.0,0.0,NaN,NaN,0,,-1.886586
coef_driving_age_child_female_N,-0.141788,0.0,0.0,NaN,NaN,0,,-0.141788
coef_driving_age_child_more_cars_than_workers_M,0.323162,0.0,0.0,NaN,NaN,0,,0.323162
...,...,...,...,...,...,...,...,...
coef_N_44,2.027136,0.0,0.0,NaN,NaN,0,,2.027136
coef_N_45,2.165050,0.0,0.0,NaN,NaN,0,,2.165050
coef_N_55,2.130772,0.0,0.0,NaN,NaN,0,,2.130772
coef_N_77,2.826423,0.0,0.0,NaN,NaN,0,,2.826423


C:\Users\davidh\AppData\Local\Temp\2\ipykernel_32752\2177651394.py:2: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  model.estimate(method='BHHH', options={'maxiter':1500})
c:\Users\davidh\.conda\envs\baydag_v2\lib\site-packages\larch\linalg\__init__.py:18: UserWarning: minimum eig 2.664521605832556e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: coef_UNAVAILABLE                                  -999.000000
┃                   coef_driving_age_child_asc_M                        -2.274643
┃                   coef_driving_age_child_asc_N                        -1.886585
┃                   coef_driving_age_child_female_N                     -0.141789
┃                   coef_driving_age_child_more_cars_than_workers_M      0.323160
┃                                                                         ...    
┃                   coef_N_44                                            2.027136
┃                   coef_N_45                                            2.165050
┃                   coef_N_55                                            2.130772
┃                   coef_N_77                                            2.826422
┃                   coef_N_88                                            2.650757
┃                   Length: 100, dtype: float64
┣          loglike: -42249.97428181838
┣        d_loglike: coef_UNAVAILABLE                                   0.000000
┃                   coef_driving_age_child_asc_M                       0.000013
┃                   coef_driving_age_child_asc_N                       0.000004
┃                   coef_driving_age_child_female_N                   -0.000005
┃                   coef_driving_age_child_more_cars_than_workers_M   -0.000006
┃                                                                        ...   
┃                   coef_N_44                                         -0.000020
┃                   coef_N_45                                         -0.000006
┃                   coef_N_55                                         -0.000050
┃                   coef_N_77                                          0.000007
┃                   coef_N_88                                         -0.000018
┃                   Length: 100, dtype: float64
┣              nit: 118
┣             nfev: 328
┣             njev: 118
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=635, microseconds=643576)
┣           method: 'slsqp'
┣          n_cases: 49762
┣ iteration_number: 118
┣          logloss: 0.8490409204175552

In [378]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
coef_UNAVAILABLE,-999.,0.00,NA,,0.00
coef_driving_age_child_asc_M,-2.27,0.374,-6.09,***,0.00
coef_driving_age_child_asc_N,-1.89,0.271,-6.96,***,0.00
coef_driving_age_child_female_N,-0.142,0.383,-0.37,,0.00
coef_driving_age_child_more_cars_than_workers_M,0.323,0.368,0.88,,0.00
coef_ft_worker_work_from_home_N,1.05,0.0409,25.65,***,0.00
coef_full_time_worker_asc_M,-6.09,0.232,-26.21,***,0.00
coef_full_time_worker_asc_N,-1.13,0.0331,-34.09,***,0.00
coef_full_time_worker_female_gender_M,0.0358,0.255,0.14,,0.00
coef_full_time_worker_female_gender_N,0.122,0.0342,3.57,***,0.00


# Output Estimation Results

In [ ]:
from larch.util.activitysim import update_coefficients
coefficients = update_coefficients(model, data)

In [ ]:
result_dir = data.edb_directory/'estimated'

In [ ]:
os.makedirs(result_dir, exist_ok=True)

### Write the re-estimated coefficients file

In [ ]:
coefficients.reset_index().to_csv(
    result_dir/"cdap_coefficients_revised.csv", 
    index=False,
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [ ]:
for hh_size, submodel in enumerate(model, 1):
    if hh_size > 2: 
        break 
        # the largest HH sizes have massive model reports that 
        # Excel doesn't handle well; review them in Jupyter 
        # instead if you are interested
    submodel.to_xlsx(
        result_dir/f"cdap_model_estimation_hhsize{hh_size}.xlsx", 
        data_statistics=True,
    )

# Next Steps

The final step is to either manually or automatically copy the `cdap_coefficients.csv_revised.csv` file to the configs folder, rename it to `cdap_coefficients.csv.csv`, and run ActivitySim in simulation mode.

In [ ]:
pd.read_csv(result_dir/"cdap_coefficients_revised.csv")